<h1><center><b>Sequence Labeling</b></center></h1>

---

# Introduction

Exploring sequence labeling problems in NLP: (i) _Part of Speech (POS) tagging;_ and (ii) _Named Entity Recognition_ (NER). Hidden Markov Model (HMM) are used first to resolve the problem of POS tagging. Conditional Random Fields (CRF) are then applied to find solutions for NER.

Some important aspects of NER are uncovered that require external (human) intervention, for example, the choice of features.

### Workflow

- Part 1: sets up the environment through installation of the required dependencies and incorporates a kernel restart.
- Part 2: downloads and explores the dataset.
- Part 3: investigates the application of Viterbi to Part of Speech tagging.
- Part 4: develops a CRF model for NER tasks.


# Part 1: Setting up the environment


In [ ]:
!pip3 install -U scikit-learn
!pip3 install git+https://github.com/MeMartijn/updated-sklearn-crfsuite.git#egg=sklearn_crfsuite
!pip3 install nltk

from IPython.core.display import HTML
HTML("<script>Jupyter.notebook.kernel.restart()</script>")


# Part 2: Getting Familiar with the Dataset

**NLTK:** There are many datasets in **Natural Language Toolkit (NLTK)** library of Python (see https://www.nltk.org). The two most widely used existing NLTK corpora are:

1. Penn TreeBank Corpus
2. CoNLL Named Entity (NE) Chunk Corpus


## Penn Treebank Corpus

> The completion of this section prepares you to answer question 1 and 2 on **Project 1 Quiz questions**.

The English Penn Treebank (PTB) corpus, and in particular the section of the corpus corresponding to the articles of _Wall Street Journal (WSJ)_, is one of the most well-known and used corpus for the evaluation of models for sequence labeling.

The formalism that underlies the POS tags assigned in the Penn Treebank is the _constituency tree_, which captures _syntactic categories_ (i.e., parts of speech) and relationships among words in a sentence.


In [ ]:
import nltk
import numpy as np
import pandas as pd
import random
import ssl
import time

from sklearn.model_selection import train_test_split

ssl._create_default_https_context = ssl._create_unverified_context

nltk.download("treebank")

from nltk.corpus import treebank

for fileid in treebank.fileids()[:5]:
    print(treebank.tagged_words(fileid))

print(len(treebank.fileids()))

[('Pierre', 'NNP'), ('Vinken', 'NNP'), (',', ','), ...]
[('Rudolph', 'NNP'), ('Agnew', 'NNP'), (',', ','), ...]
[('A', 'DT'), ('form', 'NN'), ('of', 'IN'), ...]
[('Yields', 'NNS'), ('on', 'IN'), ...]
[('J.P.', 'NNP'), ('Bolduc', 'NNP'), (',', ','), ...]
199


[nltk_data] Downloading package treebank to
[nltk_data]     /Users/krishnakodali/nltk_data...
[nltk_data]   Package treebank is already up-to-date!


### An Alternative Annotation Scheme: Universal Dependencies

An alternative format is the _dependency tree_ representation, which is intended to provide a logical form that captures semantic relationships.


### CoNLL Corpus - 2002

The CONLL-2002 dataset covers two languages: Spanish and Dutch. The Spanish dataset is a collection of newswire articles made available by the Spanish EFE News Agency. The articles are from May 2000. The tagged dataset contains words and entity tags only. The Dutch data consist of four editions of the Belgian newspaper "De Morgen" of 2000 (June 2, July 1, August 1 and September 1).

The files available in the CoNLL corpus include train and test data for the three parts of the CoNLL-2002 shared task:

1. esp.testa: Spanish test data for the development stage
2. esp.testb: Spanish test data
3. esp.train: Spanish train data
4. ned.testa: Dutch test data for the development stage
5. ned.testb: Dutch test data
6. ned.train: Dutch train data

CoNLL corpus 2002 using _ConllCorpusReader_ which is available in NLTK library. A _ConllCorpusReader_ expects a data file with the following columnn types:

COLUMN_TYPES = ('words', 'pos', 'tree', 'chunk', 'ne', 'srl', 'ignore')
where

1. 'words': column type for words
2. 'pos': column type for Part of Speech
3. 'tree': column type for parse tree
4. 'chunk': column type for short phrases present in a given structure
5. 'ne': column type for named entities
6. 'srl': column type for Semantic Role Labeling
7. 'ignore': column type which can be ignored

All data files contain a single word per line with an associated named entity tag in the IOB2 format. The IOB2 format (short for inside, outside, beginning) is a common tagging format for tagging tokens in a chunking task in computational linguistics. The _I-_ prefix before a tag indicates that the tag is inside a chunk. An _O_ tag indicates that a token belongs to no chunk. The _B-_ prefix before a tag indicates that the tag is the beginning of every chunk that immediately follows another chunk without _O_ tags between them.

For example:
Alex (_B-PER_)
is (_O_)
going (_O_)
to (_O_)
Los (_B-LOC_)
Angeles (_I-LOC_)
in (_O_)
California (_B-LOC_).

Sentence breaks are encoded as empty lines.


In [ ]:
import sklearn_crfsuite

nltk.download("conll2002")
print(nltk.corpus.conll2002.fileids())

['esp.testa', 'esp.testb', 'esp.train', 'ned.testa', 'ned.testb', 'ned.train']


[nltk_data] Downloading package conll2002 to
[nltk_data]     /Users/krishnakodali/nltk_data...
[nltk_data]   Package conll2002 is already up-to-date!


# Part 4. Part of Speech Tagging

Part-of-speech (POS) tagging is commonly used technology in Natural Language Processing that categorizes words of a text (corpus) in terms of specific parts of speech, depending on the definition of the word and its context.


In [4]:
nltk_data = list(nltk.corpus.treebank.tagged_sents())

print("DISPLAYING SENTENCES")
print(nltk_data[:5])

print("DISPLAYING WORDS")
for sent in nltk_data[:2]:
    for tuple in sent[:5]:
        print(tuple)

DISPLAYING SENTENCES
[[('Pierre', 'NNP'), ('Vinken', 'NNP'), (',', ','), ('61', 'CD'), ('years', 'NNS'), ('old', 'JJ'), (',', ','), ('will', 'MD'), ('join', 'VB'), ('the', 'DT'), ('board', 'NN'), ('as', 'IN'), ('a', 'DT'), ('nonexecutive', 'JJ'), ('director', 'NN'), ('Nov.', 'NNP'), ('29', 'CD'), ('.', '.')], [('Mr.', 'NNP'), ('Vinken', 'NNP'), ('is', 'VBZ'), ('chairman', 'NN'), ('of', 'IN'), ('Elsevier', 'NNP'), ('N.V.', 'NNP'), (',', ','), ('the', 'DT'), ('Dutch', 'NNP'), ('publishing', 'VBG'), ('group', 'NN'), ('.', '.')], [('Rudolph', 'NNP'), ('Agnew', 'NNP'), (',', ','), ('55', 'CD'), ('years', 'NNS'), ('old', 'JJ'), ('and', 'CC'), ('former', 'JJ'), ('chairman', 'NN'), ('of', 'IN'), ('Consolidated', 'NNP'), ('Gold', 'NNP'), ('Fields', 'NNP'), ('PLC', 'NNP'), (',', ','), ('was', 'VBD'), ('named', 'VBN'), ('*-1', '-NONE-'), ('a', 'DT'), ('nonexecutive', 'JJ'), ('director', 'NN'), ('of', 'IN'), ('this', 'DT'), ('British', 'JJ'), ('industrial', 'JJ'), ('conglomerate', 'NN'), ('.', '.'

## Training and Test Samples

Apply a 80:20 ratio to divide the training and test sets, and then compute the number of training and test samples for each set. Following this, the unique tags in the training data are counted.


In [ ]:
train_set, test_set = train_test_split(
    nltk_data, train_size=0.80, test_size=0.20, random_state=101
)

train_tagged_words = [tup for sent in train_set for tup in sent]
test_tagged_words = [tup for sent in test_set for tup in sent]
print(len(train_tagged_words))
print(len(test_tagged_words))

train_tagged_words[:5]

80310
20366


[('Drink', 'NN'),
 ('Carrier', 'NN'),
 ('Competes', 'VBZ'),
 ('With', 'IN'),
 ('Cartons', 'NNS')]

**Run the code below to count unique tags present in training data.**


In [6]:
tags = {tag for _, tag in train_tagged_words}
print(len(tags))
print(tags)

vocab = {word for word, _ in train_tagged_words}

45
{'CD', 'WDT', 'WP$', 'PRP$', 'TO', 'NNPS', ',', 'WP', 'RBR', '-LRB-', 'JJS', '#', ':', 'MD', 'VB', 'EX', 'VBD', 'RP', 'RBS', 'NNP', 'POS', '.', '``', '-NONE-', 'VBP', 'PRP', 'FW', 'UH', 'CC', 'IN', "''", 'DT', 'NN', 'PDT', 'VBG', '-RRB-', 'JJR', 'NNS', '$', 'WRB', 'JJ', 'VBZ', 'RB', 'VBN', 'LS'}


**45 tags in the training data**, including the -LRB- and -RRB- tags, which refer to "(" and ")" are observed respectively. The resulting number of tags suggests that 3 out of 48 tags are not present in the training corpus.


## Hidden Markov Model (HMM)

HMM is a probabilistic sequence model that leverages the principle of joint probability distribution. Two probabilities are calculated for decoding algorithm _'Viterbi Algorithm'_:

1. Emission Probability
2. Transition Probability


### Emission Probability

Emission probability, also referred to as _operational likelihood_, expresses the probability that a word is generated from a tag. First find the _tag list_ and then _count tags_ to compute the emission probability.


In [ ]:
def word_given_tag(word, tag, train_bag=train_tagged_words):
    """
    This function accepts word, tag and tagged words in training data to return count(w|tag) and count(tag)
    """
    word_tag_pairs = [(_word, _tag) for (_word, _tag) in train_bag if tag == _tag]
    return len([_word for (_word, _tag) in word_tag_pairs if _word == word]), len(
        word_tag_pairs
    )

In [ ]:
try:
    assert word_given_tag("Drink", "NN")
    assert word_given_tag("With", "IN")
    count_w_given_tag, count_tag = word_given_tag("Drink", "NN")
    print("Test Passed!")
    print(
        "The output must look like:\n The output for count_w_given_tag is:  1 \n The output for count_tag is:  10510"
    )
    print("OUTPUT:")
    print("The output for count_w_given_tag is: ", count_w_given_tag)
    print("The output for count_tag is: ", count_tag)
except AssertionError as e:
    print("Test Failed")
    raise

Test Passed!
The output must look like:
 The output for count_w_given_tag is:  1 
 The output for count_tag is:  10510
OUTPUT:
The output for count_w_given_tag is:  1
The output for count_tag is:  10510


### Transition Probability

HMM is based on the principle of random walk. An HMM-based solution obtains the probability of moving from one hidden state (tag-1 (t1)) to another hidden state (tag-2 (t2)) with a transition probability.


In [ ]:
def t2_given_t1(t2, t1, train_bag=train_tagged_words):
    """
    This function accepts two adjacent tags appearing in the text and tagged words in training data to return the count(t2|t1) and count(t1)
    """
    t2_t1_count, t1_count = 0, 0
    for index in range(len(train_bag)):
        if train_bag[index][1] == t1:
            t1_count += 1
            if index < len(train_bag) - 1 and train_bag[index + 1][1] == t2:
                t2_t1_count += 1

    return t2_t1_count, t1_count

In [ ]:
try:
    assert t2_given_t1("VBZ", "NN")
    assert t2_given_t1("NN", "NN")
    assert t2_given_t1("IN", "VBZ")
    print("Test Passed!")
    count_t2_t1, count_t1 = t2_given_t1("VBZ", "NN")
    print(
        "The output must look like:\n The output for count_t2_t1 is: 468 \n The output for count_t1 is:  10510"
    )
    print("OUTPUT:")
    print("The output for count_t2_t1 is: ", count_t2_t1)
    print("The output for count_t1 is: ", count_t1)
except AssertionError as e:
    print("Test Failed")
    raise

Test Passed!
The output must look like:
 The output for count_t2_t1 is: 468 
 The output for count_t1 is:  10510
OUTPUT:
The output for count_t2_t1 is:  468
The output for count_t1 is:  10510


### Transition Probability Matrix

A transition probability matrix (tags_matrix) is used to describe the probability of a tag, given the previous tag (to its left). Such transitions are accessed by sequence labeling processes, such as POS tagging and NER, to predict a tag for a word, given that of the previous word.

> _state probability = transition probability X emission probability_

> It might take a couple of minutes to calculate the following matrix.\_


In [ ]:
tags_matrix = np.zeros((len(tags), len(tags)), dtype="float32")
for i, t1 in enumerate(list(tags)):
    for j, t2 in enumerate(list(tags)):
        tags_matrix[i, j] = t2_given_t1(t2, t1)[0] / t2_given_t1(t2, t1)[1]

print(tags_matrix)

[[0.18421993 0.00214209 0.00035702 ... 0.00249911 0.00357015 0.        ]
 [0.00561798 0.         0.         ... 0.00561798 0.         0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]
 ...
 [0.03222996 0.00130662 0.         ... 0.06968641 0.09581882 0.        ]
 [0.01056338 0.         0.         ... 0.02288732 0.0258216  0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]]


In [ ]:
tags_df = pd.DataFrame(tags_matrix, columns=list(tags), index=list(tags))
print(tags_df.iloc[:, :10].head())

            CD       WDT       WP$      PRP$        TO      NNPS         ,  \
CD    0.184220  0.002142  0.000357  0.000357  0.025705  0.000000  0.059265   
WDT   0.005618  0.000000  0.000000  0.000000  0.000000  0.000000  0.000000   
WP$   0.000000  0.000000  0.000000  0.000000  0.000000  0.000000  0.000000   
PRP$  0.022727  0.000000  0.000000  0.000000  0.000000  0.001623  0.001623   
TO    0.074826  0.000000  0.000000  0.015661  0.000000  0.000000  0.000000   

            WP       RBR     -LRB-  
CD    0.000357  0.000714  0.001428  
WDT   0.000000  0.000000  0.000000  
WP$   0.000000  0.000000  0.000000  
PRP$  0.000000  0.000000  0.001623  
TO    0.000000  0.000580  0.000000  


## Viterbi Algorithm

The Viterbi Algorithm is a dynamic programming decoder for HMMs.


In [ ]:
def compute_state_probability(key, word, T, state):
    """
    This function accepts key, word, list of tags T, the previous state (tag) and returns the list of probabilities of each tag in T
    being the next state
    """
    state_probabilities = []
    for tag in T:
        if key == 0:
            transition_p = tags_df.loc[".", tag]
        else:
            transition_p = tags_df.loc[state[-1], tag]

        wgt = word_given_tag(word, tag)
        emission_p = wgt[0] / wgt[1]
        state_p = transition_p * emission_p
        state_probabilities.append(float(state_p))

    return state_probabilities

In [ ]:
try:
    state_probabilities = []
    state = []
    T = list(set([pair[1] for pair in train_tagged_words]))
    assert compute_state_probability(0, "At", T, state)
    state_probabilities = compute_state_probability(0, "At", T, state)
    print("Test Passed!")
    print(
        "The output must look like:\n The output for p is: [0, 0, 0, ..., 0.0003800179891361825,..., 0, 0, 0]"
    )
    print("OUTPUT:")
    print("The output for p is: ", state_probabilities)
except AssertionError as e:
    print("Test Failed")
    raise

Test Passed!
The output must look like:
 The output for p is: [0, 0, 0, ..., 0.0003800179891361825,..., 0, 0, 0]
OUTPUT:
The output for p is:  [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0003800179692916572, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]


**Define the function for Viterbi algorithm.**


In [15]:
def Viterbi(words, train_bag=train_tagged_words):
    state = []
    T = list(set([pair[1] for pair in train_bag]))

    for key, word in enumerate(words):
        p = compute_state_probability(key, word, T, state)
        pmax = max(p)

        state_max = T[p.index(pmax)]
        state.append(state_max)
    return list(zip(words, state))

**Execute this code to test the Viterbi algorithm on a few sample sentences of test dataset**


In [ ]:
random.seed(1234)  # define a random seed to get same sentences when run multiple times
random_numbers = [random.randint(1, len(test_set)) for x in range(10)]

test_subset = [test_set[i] for i in random_numbers]

test_subset_base = [tup for sent in test_subset for tup in sent]  # tagged words

test_subset_words = [tup[0] for sent in test_subset for tup in sent]  # untagged words

test_subset_tags = [
    tup[1] for sent in test_subset for tup in sent
]  # tags in test subset

## Viterbi Evaluation for POS Tagging

The Viterbi algorithm first sets up a lattice with one column for each word, and rows for tags where each cell represents each tag. Each cell $\mathrm{t_i}$ of the lattice represents the probability of that the HMM is in a given state after seeing the the previous observations (words: $\mathrm{w_1, w_2, ..., w_{i-1}}$) and passing through the most probable cell sequence $\mathrm{t_1, t_2, ..., t_{i-1}}$. The value of each cell is computed by recursively taking the most probable path (the maximum over all possible previous state sequences) that could lead us to this cell.

One way to evaluate the accuracy of sequence labeling problems is to compare the output tags (tagged_seq) for an input sequence (test_subset_words) to a human-labeled sequence that is already know to be correct (test_subset_base). Here, only 10 sentences are verified to check the accuracy as verification of the entire test set takes a very long time. Technically this notion of "accuracy" is referred to as "precision".


In [17]:
start = time.time()
tagged_seq = Viterbi(test_subset_words)
end = time.time()
difference = end - start

print("Time taken in seconds: ", difference)

check = [i for i, j in zip(tagged_seq, test_subset_base) if i == j]

accuracy = len(check) / len(tagged_seq)
print("Viterbi Algorithm Accuracy: ", accuracy * 100)

Time taken in seconds:  14.819891929626465
Viterbi Algorithm Accuracy:  90.43062200956938


The Viterbi algorithm accuracy was computed above on 10 randomly chosen sentences. Exploit the resulting tagged sequence and test\*subset_base to obtain more refined classification results, in terms of precision, recall, F1, and support.


In [ ]:
from sklearn.metrics import classification_report

actual_labels = [index for _, index in test_subset_base]
predict_labels = [index for _, index in tagged_seq]
labels = ["IN", "JJ", "NN", "VB", "RB", "DT", "CC"]

results = classification_report(
    actual_labels, predict_labels, labels=labels, zero_division=0
)
print(results)

              precision    recall  f1-score   support

          IN       1.00      1.00      1.00        16
          JJ       0.86      0.86      0.86         7
          NN       0.96      0.90      0.93        29
          VB       0.80      0.67      0.73         6
          RB       1.00      0.91      0.95        11
          DT       1.00      1.00      1.00        17
          CC       1.00      1.00      1.00         4

   micro avg       0.97      0.92      0.94        90
   macro avg       0.95      0.90      0.92        90
weighted avg       0.96      0.92      0.94        90



# Named Entity Recognition

Named-entity recognition (NER), also known as (named) entity identification, entity chunking, and entity extraction, is a subtask of information extraction that seeks to locate and classify named entities mentioned in unstructured text into predefined categories such as person names, organizations, locations.


## CoNLL2002 dataset

CoNLL2002 corpus will be used for the NER task that was downloaded earlier. There are as many as 6 fileids available in the dataset. In each file, every line represents a tagged word seperated by an empty space as _'word pos_tag NER_tag'_. Thus, three columns in every line consists of

1. The first column: Word
2. The second column: POS tag
3. The third column: Named entity

The data are represented with BIO tagging.

For this, let's focus on the simpler BIO tagging scheme: any token that begins a span of interest is labeled B, tokens that occur inside a span are labeled I, and any tokens outside of any span of interest are labeled O. Also, labels are augmented with named-entity tags:

1. 'B-LOC' : beginning location
2. 'B-MISC': beginning miscellaneous
3. 'B-ORG' : beginning organization
4. 'B-PER' : beginning person
5. 'I-LOC' : inside location
6. 'I-MISC': inside miscellaneous
7. 'I-ORG' : inside organization
8. 'I-PER' : inside person
9. 'O'. : Outside of span of interest


In [23]:
print(nltk.corpus.conll2002.raw("esp.testa")[:100])

Sao NC B-LOC
Paulo VMI I-LOC
( Fpa O
Brasil NC B-LOC
) Fpt O
, Fc O
23 Z O
may NC O
( Fpa O
EFECOM N


## Training and Test Samples


Train and test a NER model using Spanish data from the CoNLL2002 dataset. The model is trained on _'esp.train'_ and tested on _'esp.testb'_ Dataset is extracted into two variables, _train_sents_ and _test_sents_, respectively. The _iob_sents_ function extracts the raw text of file in the form of a list of _tuples <word, pos_tag, ner_tag>_ for each sentence.


In [24]:
train_sents = list(nltk.corpus.conll2002.iob_sents("esp.train"))
test_sents = list(nltk.corpus.conll2002.iob_sents("esp.testb"))

train_sents[0]

[('Melbourne', 'NP', 'B-LOC'),
 ('(', 'Fpa', 'O'),
 ('Australia', 'NP', 'B-LOC'),
 (')', 'Fpt', 'O'),
 (',', 'Fc', 'O'),
 ('25', 'Z', 'O'),
 ('may', 'NC', 'O'),
 ('(', 'Fpa', 'O'),
 ('EFE', 'NC', 'B-ORG'),
 (')', 'Fpt', 'O'),
 ('.', 'Fp', 'O')]

In [25]:
def compute_tag_distribution(sents):
    """
    This function accepts a tuple <word, pos, ner> to return the NER_tag frequency distribution
    """
    ner_tag_frequency = dict()
    for sent in sents:
        for word, _, ner in sent:
            if ner not in ner_tag_frequency:
                ner_tag_frequency[ner] = 1
            else:
                ner_tag_frequency[ner] += 1

    return ner_tag_frequency

In [26]:
try:
    assert compute_tag_distribution(train_sents)
    assert compute_tag_distribution(test_sents)
    train_ner_tag_frequency = compute_tag_distribution(train_sents)
    test_ner_tag_frequency = compute_tag_distribution(test_sents)
    print("Test Passed!")
    print(
        "The output for training corpora must look like: {'B-LOC': 4913, 'O': 231920, 'B-ORG': 7390, 'B-PER': 4321, ...} "
    )
    print("OUTPUT for train_sents ner_tag_frequency is:", train_ner_tag_frequency)
    print(
        "The output for test corpora must look like: {'B-LOC': 1084, 'I-LOC': 325, 'O': 45355, 'B-ORG': 1400, ...} "
    )
    print("OUTPUT for test_sents ner_tag_frequency is:", test_ner_tag_frequency)
except AssertionError as e:
    print("Test Failed")
    raise

Test Passed!
The output for training corpora must look like: {'B-LOC': 4913, 'O': 231920, 'B-ORG': 7390, 'B-PER': 4321, ...} 
OUTPUT for train_sents ner_tag_frequency is: {'B-LOC': 4913, 'O': 231920, 'B-ORG': 7390, 'B-PER': 4321, 'I-PER': 3903, 'B-MISC': 2173, 'I-ORG': 4992, 'I-LOC': 1891, 'I-MISC': 3212}
The output for test corpora must look like: {'B-LOC': 1084, 'I-LOC': 325, 'O': 45355, 'B-ORG': 1400, ...} 
OUTPUT for test_sents ner_tag_frequency is: {'B-LOC': 1084, 'I-LOC': 325, 'O': 45355, 'B-ORG': 1400, 'B-MISC': 339, 'B-PER': 735, 'I-PER': 634, 'I-ORG': 1104, 'I-MISC': 557}


The NER tag distribution differs due to the size difference between the training and test corpora. However, the distributions follows a similar trend, with the highest number of tags being "O", etc. in both distributions.


## Feature Extraction

As a starting point for the named-entity recognition task, features are extracted for every word. A feature is an individual measurable property or characteristic of a word, for example, its part of speech (e.g., the POS for "eat" is a Verb). Choosing informative, discriminating and independent features is crucial for development of effective algorithms in pattern recognition, classification and regression. Producing accurate output for sequence labeling relies crucially on careful selection of features.

In addition to POS tags, other types of features may be extracted, e.g., word parts, simplified POS tags, lower/title/upper case flags, and features of nearby words. To employ all such features in the task of Named Entity recognition, covert them into sklearn-crfsuite format. Each sentence is converted into a list of dicts, i.e., a dictionary that maps feature names to feature indices which is a very simple baseline.


The following feature extraction steps are added for a given word and its neighbors (steps 1, 2, 3, and 4 refer to boolean values and 5 refers to a multi-category POS):

1. Word is lower-case (non-capitalized letters, e.g., "a")
2. Case of a word is `upper' (capitalized letters, e.g., "A")
3. Word is title (e.g., "Mr.")
4. Word is a digit (e.g., "10")
5. POS tag of word (e.g., "JJ")
6. Repeat steps from 2 to 5 for its neighbors

Tag frequency may influence the feature vector. Thus, features need to be weighted according to the number of times their corresponding tag appears. This introduces 'bias', an independent feature, which balances out the influence of varying tag frequency in the training corpus. Using 'bias' is a prevailing approach to address _feature bias_ in adjusting the training loss for manually designed biased features. However currently, simplify assumption of uniform NER tag frequency distribution during model, e.g., ORG is assumed to occur as frequently as PERS.


In [33]:
def word2features(sent, i):
    """
    This function accepts a sent (list of tuple in sentence) to return the additional features.
    """

    word = sent[i][0]
    postag = sent[i][1]

    features = {
        "bias": 1.0,
        "word.lower()": word.lower(),
        "word[-3:]": word[-3:],
        "word[-2:]": word[-2:],
        "word.isupper()": word.isupper(),
        "word.istitle()": word.istitle(),
        "word.isdigit()": word.isdigit(),
        "postag": postag,
        "postag[:2]": postag[:2],
    }
    if i > 0:
        word1 = sent[i - 1][0]
        postag1 = sent[i - 1][1]

        features.update(
            {
                "-1:word.lower()": word1.lower(),
                "-1:word.istitle()": word1.istitle(),
                "-1:word.isupper()": word1.isupper(),
                "-1:postag": postag1,
                "-1:postag[:2]": postag1[:2],
            }
        )
    else:
        features["BOS"] = True

    if i < len(sent) - 1:
        word1 = sent[i + 1][0]
        postag1 = sent[i + 1][1]

        features.update(
            {
                "+1:word.lower()": word1.lower(),
                "+1:word.istitle()": word1.istitle(),
                "+1:word.isupper()": word1.isupper(),
                "+1:postag": postag1,
                "+1:postag[:2]": postag1[:2],
            }
        )
    else:
        features["EOS"] = True

    return features

In [28]:
try:
    assert word2features(train_sents[3], 2)
    features = word2features(train_sents[3], 2)
    print("Test Passed!")
    print(
        "The output must look like: {'bias': 1.0, 'word.lower()': 'del', 'word[-3:]': 'del', ..., }"
    )
    print("OUTPUT:")
    print(features)
except AssertionError as e:
    print("Test Failed")
    raise

Test Passed!
The output must look like: {'bias': 1.0, 'word.lower()': 'del', 'word[-3:]': 'del', ..., }
OUTPUT:
{'bias': 1.0, 'word.lower()': 'del', 'word[-3:]': 'del', 'word[-2:]': 'el', 'word.isupper()': False, 'word.istitle()': False, 'word.isdigit()': False, 'postag': 'SP', 'postag[:2]': 'SP', '-1:word.lower()': 'petición', '-1:word.istitle()': False, '-1:word.isupper()': False, '-1:postag': 'NC', '-1:postag[:2]': 'NC', '+1:word.lower()': 'abogado', '+1:word.istitle()': True, '+1:word.isupper()': False, '+1:postag': 'NC', '+1:postag[:2]': 'NC'}


In [29]:
def sent2features(sent):
    return [word2features(sent, i) for i in range(len(sent))]


def sent2labels(sent):
    return [label for token, postag, label in sent]


def sent2tokens(sent):
    return [token for token, postag, label in sent]

### Extracting features

The input to feature extraction is a sentence. The characteristics for each word in a sentence are obtained as a set of _features_ in the form of a dictionary, as illustrated in Part 3. Will be using sent2feature and sent2labels for feature extraction at the sentence level, recursively extracting features at the word level using the word2feature function.


In [30]:
X_train = [sent2features(s) for s in train_sents]
y_train = [sent2labels(s) for s in train_sents]

X_test = [sent2features(s) for s in test_sents]
y_test = [sent2labels(s) for s in test_sents]

X_train[0][3]

{'bias': 1.0,
 'word.lower()': ')',
 'word[-3:]': ')',
 'word[-2:]': ')',
 'word.isupper()': False,
 'word.istitle()': False,
 'word.isdigit()': False,
 'postag': 'Fpt',
 'postag[:2]': 'Fp',
 '-1:word.lower()': 'australia',
 '-1:word.istitle()': True,
 '-1:word.isupper()': False,
 '-1:postag': 'NP',
 '-1:postag[:2]': 'NP',
 '+1:word.lower()': ',',
 '+1:word.istitle()': False,
 '+1:word.isupper()': False,
 '+1:postag': 'Fc',
 '+1:postag[:2]': 'Fc'}

## Conditional Random Field (CRF) model

A Conditional Random Field (CRF) is a standard model for predicting the most likely sequence of labels that correspond to a sequence of inputs. CRF is a labeler in which the tag of the present word (denoted as yᵢ) depends only on the tag of just the previous word(denoted by yᵢ₋₁).


In [31]:
crf = sklearn_crfsuite.CRF(
    algorithm="lbfgs",
    c1=1.0,
    c2=1e-3,
    max_iterations=20,
    all_possible_transitions=True,
)
crf.fit(X_train, y_train)

CRF(algorithm='lbfgs', all_possible_transitions=True, c1=1.0, c2=0.001,
    max_iterations=20)

## CRF Evaluation for Named Entity Recognition


In [32]:
from sklearn_crfsuite.metrics import flat_classification_report
import numpy as np

y_pred = crf.predict(X_test)

result = flat_classification_report(y_test, y_pred)
print(result)

              precision    recall  f1-score   support

       B-LOC       0.67      0.61      0.64      1084
      B-MISC       0.35      0.10      0.15       339
       B-ORG       0.66      0.76      0.71      1400
       B-PER       0.74      0.77      0.75       735
       I-LOC       0.49      0.24      0.32       325
      I-MISC       0.55      0.20      0.30       557
       I-ORG       0.63      0.79      0.70      1104
       I-PER       0.81      0.89      0.85       634
           O       0.99      1.00      0.99     45355

    accuracy                           0.95     51533
   macro avg       0.66      0.59      0.60     51533
weighted avg       0.95      0.95      0.95     51533

